In [1]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
root_dir   = '/user_data/mmhender/imStat'
sys.path.append(os.path.join(root_dir,'code'))
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_fit_old
from model_src import fwrf_predict as fwrf_predict
from model_src import fwrf_predict_old
from model_src import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features
from model_src import texture_statistics_pyramid_old, bdcn_features_old
from utils import nsd_utils, roi_utils

bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'

from model_fitting import initialize_fitting, merge_features

fpX = np.float32
device = initialize_fitting.init_cuda()

#device: 1
device#: 0
device name: GeForce RTX 2080 Ti

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32


In [24]:
fitting_type='bdcn'
# fitting_type='pyramid'
subject=1
volume_space = False
up_to_sess = 1
n_ori = 4
n_sf = 4
nonlin_fn = False
padding_mode = 'circular'
include_simple = True;include_complex = True; \
include_autocorrs = True; include_crosscorrs = True; group_all_hl_feats = False; \
sample_batch_size = 50; voxel_batch_size = 100; \
zscore_features = True; ridge = True; \
shuffle_images = False; random_images = False; random_voxel_data = False; \
do_fitting = True; do_val = True; do_varpart = True; date_str = None;
shuff_rnd_seed = 0; debug = True; \
do_pca = False; min_pct_var = 99; max_pc_to_retain = 400; map_ind = -1; \
n_prf_sd_out = 2; mult_patch_by_prf = True; \
downsample_factor = 1.0; do_nms = False

In [25]:

if 'pyramid' in fitting_type:
    model_name, feature_types_exclude = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)

    if 'plus_bdcn' in fitting_type:
        model_name2 = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
        model_name = model_name + '_plus_' + model_name2
#         save_all = save_fn_pyramid_bdcn
#     else:
#         save_all = save_fn_pyramid

elif 'bdcn' in fitting_type:
    model_name = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
#     save_all = save_fn_bdcn

elif 'gabor' in fitting_type:
    model_name, feature_types_exclude = initialize_fitting.get_model_name(ridge, n_ori, n_sf, include_pixel, include_simple, include_complex, include_autocorrs, include_crosscorrs)
#     save_all = save_fn_gabor


output_dir, fn2save = initialize_fitting.get_save_path(root_dir, subject, volume_space, model_name, shuffle_images, random_images, random_voxel_data, debug, date_str)

# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

sessions = np.arange(0,up_to_sess)
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, image_order = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                     voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                      zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                      shuffle_images=shuffle_images, random_images=random_images, \
                                                                                         random_voxel_data=random_voxel_data)
    

Time Stamp: Sep-07-2021_2351_32

Will save final output file to /user_data/mmhender/model_fits/S01_surface/bdcn_fused/Sep-07-2021_2351_32_DEBUG/


Surface space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/freesurfer/subj01/label

18511 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[ 0. 26. 27. 28. 30. 31. 32. 33.]
7047 voxels are defined (differently) in both retinotopic areas and category areas

66545 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 

In [4]:
if 'pyramid' in fitting_type:
    # Need a multiple of 8
    process_at_size=240
    trn_stim_data = skimage.transform.resize(trn_stim_data, output_shape=(trn_stim_data.shape[0],1,process_at_size, process_at_size))
    val_stim_data = skimage.transform.resize(val_stim_data, output_shape=(val_stim_data.shape[0],1,process_at_size, process_at_size))

In [5]:
# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)
# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    



Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]
most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [6]:
# Set up the contour feature extractor
pretrained_model_file = os.path.join(bdcn_path,'params','bdcn_pretrained_on_bsds500.pth')
_feature_extractor = bdcn_features.bdcn_feature_extractor(pretrained_model_file, device, aperture_rf_range, n_prf_sd_out, \
                                           batch_size=10, map_ind=map_ind, mult_patch_by_prf=mult_patch_by_prf,
                                        downsample_factor = downsample_factor, do_nms = do_nms, \
                                             do_pca = do_pca, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain)

/user_data/mmhender/toolboxes/BDCN/bdcn.py:232: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.080)


In [7]:
_feature_extractor_old = bdcn_features_old.bdcn_feature_extractor(pretrained_model_file, device, aperture_rf_range, n_prf_sd_out, \
                                           batch_size=10, map_ind=map_ind, mult_patch_by_prf=mult_patch_by_prf,
                                        downsample_factor = downsample_factor, do_nms = do_nms, \
                                             )

In [8]:
# add an intercept
add_bias=True
# determines whether to shuffle before separating the nested heldout data for lambda and param selection. 
# always using true.
shuffle=True 

In [10]:
shuff_rnd_seed = 92207
best_losses, best_lambdas, best_params = fwrf_fit.fit_fwrf_model(trn_stim_data, trn_voxel_data, _feature_extractor, models, \
                                               lambdas, zscore=zscore_features, add_bias=add_bias, \
                                               voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
                                               shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, \
                                                                               debug=debug)

dtype = <class 'numpy.float32'>
device = cuda:0
trn_size = 619 (90.0%)
Seeding random number generator: seed is 92207
Initializing for fitting: finding the size of feature matrix for each candidate prf
Clearing BDCN contour features from memory.
---------------------------------------


Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(688, 1, 227, 227)
Final array shape is:
torch.Size([688, 1, 227, 227])
time elapsed = 9.38513
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 8]
[min max] of first image patch is:
[tensor(0.0199, device='cuda:0'), tensor(0.5242, device='cuda:0')]

Fitting version 0 of 1: full_model, 
fitting model    0 of 875 , voxels [ 14900:14912 ] of 14913
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
Getting features for pRF [x,y,sigma]:
[-0.49210526315789477, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227

In [11]:
shuff_rnd_seed = 92207
best_losses_old, best_lambdas_old, best_params_old, pc = fwrf_fit_old.fit_bdcn_model(trn_stim_data, trn_voxel_data, _feature_extractor_old, models, \
                                               lambdas, zscore=zscore_features, add_bias=add_bias, \
                                               voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
                                               shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, \
                                                                                 debug=debug, \
                                            do_pca = do_pca, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain)


trn_size = 619 (90.0%)
dtype = <class 'numpy.float32'>
device = cuda:0
---------------------------------------
Seeding random number generator: seed is 92207
Clearing BDCN contour features from memory.


Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(688, 1, 227, 227)
Final array shape is:
torch.Size([688, 1, 227, 227])
time elapsed = 8.72670
pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 8]
[min max] of first image patch is:
[tensor(0.0199, device='cuda:0'), tensor(0.5242, device='cuda:0')]
Original dims of training features:
(619, 64)
fitting model    0 of 875 , voxels [ 14900:14912 ] of 14913
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
pRF [x,y,sigma]:
[-0.49210526315789477, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 21]
[min max] of first image patch is:
[tensor(0.0260, device='cuda:0'), tensor(0.3674, device='cuda:0')]
Origina

In [12]:
val_cc, val_r2 = fwrf_predict.validate_fwrf_model(best_params, models, val_voxel_data, val_stim_data, _feature_extractor, \
                                   sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug, dtype=fpX)




Clearing BDCN contour features from memory.
Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(62, 1, 227, 227)
Final array shape is:
torch.Size([62, 1, 227, 227])
time elapsed = 0.58118
Getting features for pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 8]
[min max] of first image patch is:
[tensor(0.0335, device='cuda:0'), tensor(0.3218, device='cuda:0')]
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
Getting features for pRF [x,y,sigma]:
[-0.49210526315789477, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 21]
[min max] of first image patch is:
[tensor(0.0378, device='cuda:0'), tensor(0.1918, device='cuda:0')]
Clearing BDCN contour features from memory.
Getting predictions for voxels [0-99] of 14913

Evaluating version 0 of 1: full_model
Includes 51529 features
number of zeros:
51465
size of weights is:
torch.Size([100, 51529])
Getting pre

In [23]:
thing = {'a': 2, 'b': 3}
thing2 = {'c': 5, 'd':60}

thing.update(thing2)

thing

{'a': 2, 'b': 3, 'c': 5, 'd': 60}

In [14]:
val_cc_old, val_r2_old = fwrf_predict_old.validate_bdcn_model(best_params_old, models, val_voxel_data, val_stim_data, _feature_extractor_old, \
                                   sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug, dtype=fpX)


starting validation function
starting to initialize big arrays
about to start loop
Clearing BDCN contour features from memory.
Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running BDCN contour feature extraction...
Images array shape is:
(62, 1, 227, 227)
Final array shape is:
torch.Size([62, 1, 227, 227])
time elapsed = 0.54893
pRF [x,y,sigma]:
[-0.55, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 8]
[min max] of first image patch is:
[tensor(0.0335, device='cuda:0'), tensor(0.3218, device='cuda:0')]
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
pRF [x,y,sigma]:
[-0.49210526315789477, -0.55, 0.03999999910593033]
bbox to crop is:
[219, 227, 0, 21]
[min max] of first image patch is:
[tensor(0.0378, device='cuda:0'), tensor(0.1918, device='cuda:0')]
Clearing BDCN contour features from memory.
Getting predictions for voxels [0-99] of 14913
size of feature matrix to use is:
(62, 168, 100)
Getting predictions for voxels [100-199] of 14913
si

In [17]:
val_cc[0:10]

array([[-0.21275681],
       [ 0.1307291 ],
       [ 0.20311193],
       [-0.14022963],
       [ 0.16126229],
       [ 0.17579162],
       [ 0.19388685],
       [ 0.32910824],
       [ 0.05345447],
       [-0.07433432]], dtype=float32)

In [18]:
val_cc_old[0:10]

array([[-0.21275684],
       [ 0.13072911],
       [ 0.20311193],
       [-0.14022975],
       [ 0.1612623 ],
       [ 0.17579159],
       [ 0.19388682],
       [ 0.3291083 ],
       [ 0.05345447],
       [-0.07433435]], dtype=float32)

In [19]:
val_r2[0:10]

array([[-0.08576927],
       [ 0.01187935],
       [-0.03130872],
       [-0.41322434],
       [ 0.02266493],
       [-0.00404857],
       [-0.00952173],
       [ 0.05535273],
       [ 0.00231291],
       [-0.07745466]], dtype=float32)

In [20]:
val_r2_old[0:10]

array([[-0.0857693 ],
       [ 0.01187936],
       [-0.03130872],
       [-0.41322443],
       [ 0.02266494],
       [-0.00404855],
       [-0.00952173],
       [ 0.05535276],
       [ 0.00231291],
       [-0.07745465]], dtype=float32)

In [6]:
# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height=n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, feature_types_exclude=feature_types_exclude, n_prf_sd_out=n_prf_sd_out, aperture=aperture, device=device, \
                              group_all_hl_feats = group_all_hl_feats, do_varpart = do_varpart)
feature_info = [_feature_extractor.feature_column_labels, _feature_extractor.feature_types_include]


[]


In [8]:
# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid_old.steerable_pyramid_extractor(pyr_height=n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor_old = texture_statistics_pyramid_old.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, \
                                                                                  feature_types_exclude=feature_types_exclude, \
                                                                                  n_prf_sd_out=n_prf_sd_out, aperture=aperture, \
                                                                                  device=device)

[]


In [19]:
best_params[1].shape
best_params[1][0,0:10,0]

array([ 6.4805418e-04, -4.2727246e-05,  2.0694181e-03,  4.6552962e-04,
        1.5816985e-03, -8.3285588e-04,  4.8995239e-04, -1.8527535e-04,
       -1.3699161e-04,  5.1099481e-04], dtype=float32)

In [23]:
best_params[1].shape
best_params[1][0,0:10,1]

array([ 0.00049042,  0.00017432,  0.00223572,  0.00060843,  0.00142537,
       -0.00067342,  0.        ,  0.        ,  0.        ,  0.        ],
      dtype=float32)

In [21]:
best_params_old[1].shape
best_params_old[1][0,0:10,0]

array([ 6.4805418e-04, -4.2727246e-05,  2.0694181e-03,  4.6552962e-04,
        1.5816985e-03, -8.3285588e-04,  4.8995239e-04, -1.8527535e-04,
       -1.3699161e-04,  5.1099481e-04], dtype=float32)

In [27]:
best_params_old[1][0,0:10,4]

array([ 0.00304986, -0.00044732,  0.00770182,  0.0013891 ,  0.00638007,
       -0.00309239,  0.0003045 , -0.0017243 , -0.00039225,  0.00087905],
      dtype=float32)

In [29]:
best_losses[0:10,0]

array([38.840324, 56.365883, 64.80278 , 82.17668 , 57.838043, 80.0087  ,
       69.31924 , 69.64662 , 59.50539 , 73.70018 ], dtype=float32)

In [31]:
best_losses_old[0:10,0]

array([38.840324, 56.365883, 64.80278 , 82.17668 , 57.838043, 80.0087  ,
       69.31924 , 69.64662 , 59.50539 , 73.70018 ], dtype=float32)

In [32]:
best_lambdas[0:10,0]

array([7, 8, 5, 8, 4, 3, 4, 4, 4, 6])

In [34]:
best_lambdas_old[0:10,0]

array([7, 8, 5, 8, 4, 3, 4, 4, 4, 6])

In [15]:
best_losses_old[0:10]

array([38.542297, 54.950672, 66.98472 , 77.628006, 56.934196, 87.33652 ,
       74.53955 , 70.959175, 61.502754, 72.14632 ], dtype=float32)

In [39]:
best_params_old[5].shape

(14913, 15)

In [36]:
best_params[5][0:10,0]

array([1, 1, 0, 1, 1, 0, 0, 0, 1, 0])

In [37]:
best_params_old[5][0:10,0]

array([1, 1, 0, 1, 1, 0, 0, 0, 1, 0])

In [40]:
best_params[2][0:10,0]

array([ 4.1831576e-05, -8.7276574e-05,  2.4075587e-03,  3.4289420e-05,
        1.7660893e-03,  1.9703126e-03, -1.7527193e-03, -2.4655187e-03,
       -5.8551114e-03,  1.7888251e-03], dtype=float32)

In [42]:
best_params_old[2][0:10,0]

array([ 4.1831576e-05, -8.7276574e-05,  2.4075587e-03,  3.4289420e-05,
        1.7660893e-03,  1.9703126e-03, -1.7527193e-03, -2.4655187e-03,
       -5.8551114e-03,  1.7888251e-03], dtype=float32)

In [43]:
best_params[0][0:10,0,:]

array([[-0.49210526, -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ]])

In [44]:
best_params_old[0][0:10,0,:]

array([[-0.49210526, -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ],
       [-0.49210526, -0.55      ,  0.04      ],
       [-0.55      , -0.55      ,  0.04      ]])

In [45]:
best_params[4][0,0:10]

array([2.3927572e-01, 1.7860206e-02, 3.0109861e+00, 1.1965325e+02,
       3.3682201e-02, 5.7449959e-02, 2.5761364e+00, 1.9755992e+00,
       2.8495173e+00, 1.8524592e+00], dtype=float32)

In [46]:
best_params_old[4][0,0:10]

array([2.3927572e-01, 1.7860206e-02, 3.0109861e+00, 1.1965325e+02,
       3.3682201e-02, 5.7449959e-02, 2.5761364e+00, 1.9755992e+00,
       2.8495173e+00, 1.8524592e+00], dtype=float32)

In [14]:
shuff_rnd_seed = 92207
best_losses_old, best_lambdas_old, best_params_old, pc = fwrf_fit_old.fit_texture_model_ridge(trn_stim_data, trn_voxel_data, _feature_extractor_old, models, \
                                               lambdas, zscore=zscore_features, add_bias=add_bias, \
                                               voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
                                               shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, \
                                                                                 debug=debug, do_varpart=True)


trn_size = 619 (90.0%)
dtype = <class 'numpy.float32'>
device = cuda:0
---------------------------------------
Seeding random number generator: seed is 92207
Clearing steerable pyramid features from memory.


Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Running steerable pyramid feature extraction...
Images array shape is:
(688, 1, 240, 240)
time elapsed = 123.10804
Computing higher order correlations...
time elapsed = 8.82800
Final size of features concatenated is [688 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']

Fitting version 0 of 15: full_model, 
Voxels

In [22]:
pc[0]

array([ 0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5